# **DB 생성 및 관리**

In [1]:
import pandas as pd
import sqlite3
import os
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [26]:
path = os.getcwd() + '/testdb.db'
conn = sqlite3.connect(path)

### (1) DB 생성

In [31]:
path = os.getcwd() + '/db/em.db'
conn = sqlite3.connect(path)

In [43]:
conn = sqlite3.connect(path)
cursor = conn.cursor()

# log 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS log (
    datetime TEXT NOT NULL, 
    input_text TEXT NOT NULL, 
    input_latitude REAL NOT NULL,
    input_longitude REAL NOT NULL,
    em_class INTEGER NOT NULL, 
    hospital1 TEXT,
    addr1 TEXT ,
    tel1 TEXT , 
    hospital2 TEXT ,
    addr2 TEXT , 
    tel2 TEXT , 
    hospital3 TEXT , 
    addr3 TEXT , 
    tel3 TEXT,
    comment 
)
''')

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

In [6]:
# ① 연결
db_path = os.getcwd() + '/db/em.db'
conn = sqlite3.connect(db_path)

# ② 작업 : select
df_db = pd.read_sql('SELECT * FROM log', conn)
display(df_db)

# ③ 연결 종료
conn.close()

,datetime,input_text,input_latitude,input_longitude,em_class,hospital1,addr1,tel1,hospital2,addr2,tel2,hospital3,addr3,tel3,comment
0,2024-12-18 14:27:14,"{""summary"": ""낙상 후 머리부상, 피 흘림, 어지러움 증상"", ""부위"": ...",37.358618,127.115036,2,분당서울대학교병원,None,031-787-2114,대진의료재단분당제생병원,None,031-779-0114,국군수도병원,None,031-725-6075,None
1,2024-12-18 14:27:14,"{""summary"": ""낙상 후 머리부상, 피 흘림, 어지러움 증상"", ""부위"": ...",37.358618,127.115036,2,분당서울대학교병원,"경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)",031-787-2114,대진의료재단분당제생병원,경기도 성남시 분당구 서현로180번길 20 (서현동),031-779-0114,국군수도병원,경기도 성남시 분당구 새마을로177번길 81 (율동),031-725-6075,None
2,2024-12-18 14:27:14,"{""summary"": ""낙상 후 머리부상, 피 흘림, 어지러움 증상"", ""부위"": ...",37.358618,127.115036,2,분당서울대학교병원,"경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)",031-787-2114,대진의료재단분당제생병원,경기도 성남시 분당구 서현로180번길 20 (서현동),031-779-0114,국군수도병원,경기도 성남시 분당구 새마을로177번길 81 (율동),031-725-6075,None


In [45]:
# result = cursor.fetchall()

### (3) log 테이블 입력 함수

* 현재 일시 저장하기 예제
 * 현재 연월일 시분초 뽑기
- from datetime import datetime
- dt = datetime.now()
- dt = dt.strftime('%Y-%m-%d %H:%M:%S')

In [3]:
from datetime import datetime
dt = datetime.now()
dt = dt.strftime('%Y-%m-%d %H:%M:%S')
dt

'2024-12-18 14:54:47'

# 정보 받아오기

In [4]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import sys
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# 조에서 생성한 모듈 불러오기 -------------
import emergency as em

## path 설정

In [7]:
path = os.getcwd() + "/"
sys.path.append(path)
audio_path = path + 'audio/'
save_directory = path + 'fine_tuned_bert_our/'

## 파이프라인 가동

In [8]:
#현재 존재하는 응급실 정보, 기본 정보 받아오기
emergency = pd.read_csv(path + '응급실 정보.csv')

#api key 설정
openai.api_key = em.load_file(path + '/api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

# audio -> text 변환, 예측하기
df = em.audio2text(audio_path)
df = em.text2summary(df)
pred = em.model_prediction(save_directory, df)

# 위치정보 받아온거 합치기(df)
df_location = pd.read_excel(path + "audio_location.xlsx")
df_location.rename(columns={'filename': 'Audio'}, inplace=True)
df = pd.merge(df, df_location, on='Audio', how='left')

#추천 병원 3개
em.recom_em(path, df)

탐색범위:11.0km * 11.0km


[(2626, '분당서울대학교병원', '031-787-2114', '031-787-3119'),
 (4707, '대진의료재단분당제생병원', '031-779-0114', '031-779-0119'),
 (5973, '국군수도병원', '031-725-6075', '031-725-6119')]

In [9]:
# module recom_em return값 튜플로 받게 수정
# 3개의 값 저장
result = em.recom_em(path, df)

탐색범위:11.0km * 11.0km


In [11]:
addr = []
for i in range(len(result)):
    hospital_name = result[i][1]
    addr.append(emergency.loc[emergency['병원이름'] == hospital_name]['주소'].iloc[0])
addr

['경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)',
 '경기도 성남시 분당구 서현로180번길 20 (서현동)',
 '경기도 성남시 분당구 새마을로177번길 81 (율동)']

In [13]:
# 연결
db_path = os.getcwd() + '/db/em.db'
conn = sqlite3.connect(db_path)

#SQL 쿼리문 작성
insert_query = """
INSERT INTO log(datetime, input_text, input_latitude, input_longitude, em_class,
                hospital1, addr1, tel1, 
                hospital2,addr2, tel2, 
                hospital3, addr3, tel3) 
VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)
"""
# INSERT INTO log VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)로 해도 작동한다. 
cursor = conn.cursor()

# 주소를 제외한 나머지만 넘김
cursor.execute(insert_query, 
               (dt, df['summary'][0], df['위도'][0], df['경도'][0], pred, 
                result[0][1], addr[0], result[0][2], 
                result[1][1], addr[1], result[1][2], 
                result[2][1], addr[2], result[2][2]
               )
              )

conn.commit()
# 연결 종료
conn.close()

In [14]:
# 연결
conn = sqlite3.connect(db_path)

# 작업 : select
df_db = pd.read_sql('SELECT * FROM log', conn)
display(df_db)

# 연결 종료
conn.close()

,datetime,input_text,input_latitude,input_longitude,em_class,hospital1,addr1,tel1,hospital2,addr2,tel2,hospital3,addr3,tel3,comment
0,2024-12-18 14:27:14,"{""summary"": ""낙상 후 머리부상, 피 흘림, 어지러움 증상"", ""부위"": ...",37.358618,127.115036,2,분당서울대학교병원,None,031-787-2114,대진의료재단분당제생병원,None,031-779-0114,국군수도병원,None,031-725-6075,None
1,2024-12-18 14:27:14,"{""summary"": ""낙상 후 머리부상, 피 흘림, 어지러움 증상"", ""부위"": ...",37.358618,127.115036,2,분당서울대학교병원,"경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)",031-787-2114,대진의료재단분당제생병원,경기도 성남시 분당구 서현로180번길 20 (서현동),031-779-0114,국군수도병원,경기도 성남시 분당구 새마을로177번길 81 (율동),031-725-6075,None
2,2024-12-18 14:27:14,"{""summary"": ""낙상 후 머리부상, 피 흘림, 어지러움 증상"", ""부위"": ...",37.358618,127.115036,2,분당서울대학교병원,"경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)",031-787-2114,대진의료재단분당제생병원,경기도 성남시 분당구 서현로180번길 20 (서현동),031-779-0114,국군수도병원,경기도 성남시 분당구 새마을로177번길 81 (율동),031-725-6075,None
3,2024-12-18 14:54:47,"{""summary"": ""노인이 계단에서 쓰러지고 머리에 상처가 있으며 피가 납니다....",37.358618,127.115036,2,분당서울대학교병원,"경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)",031-787-2114,대진의료재단분당제생병원,경기도 성남시 분당구 서현로180번길 20 (서현동),031-779-0114,국군수도병원,경기도 성남시 분당구 새마을로177번길 81 (율동),031-725-6075,None


In [ ]:
# 위도, 경도를 계속해서 입력해주는 함수
if pred < 3 :
   df['위도'], df['경도'] = map(float, input().split())
   em.recom_em(path, df)
else : print("집에서 휴식을 취하세요")